### Use checkV instead of cd-hit
Problems with cdhit not dereplicating everything properly, so would like to try checkVs scripts instead.

If the dataset too big: Split into smaller, then later merge. 


In [ ]:
# make a blastdb
# in: /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/viral
srun --account=ctbrowngrp -p med2 -J checkv -t 5:00:00 -c 1 --mem=50gb --pty bash

mamba activate checkv
makeblastdb -in 250418_viral_sequences.genomad.fa -dbtype nucl -out blastdb/250418_viral_sequences


In [ ]:
# blast doesnt do multithreading but we can use parallel
# make smaller files
splits=500
seqkit split ../250418_viral_sequences.genomad.fa --by-part $splits --out-dir sequence_split


In [ ]:
srun --account=ctbrowngrp -p med2 -J checkv -t 5:00:00 -c 2 --mem=150gb --pty bash
module load parallel
mamba activate checkv

ls sequence_split | parallel "
    blastn -query sequence_split/{} \
    -db ../blastdb/250418_viral_sequences \
    -outfmt '6 std qlen slen' \
    -max_target_seqs 10000 \
    -out blastout/{} \
    -num_threads 25
"

## Use anicalc
cat my_blast/* > my_blast.tsv
../anicalc.py -i 250515_blastout.tsv -o 250515_vir_ani.tsv

# and aniclust
../aniclust.py --fna \
../250418_viral_sequences.genomad.fa --ani 250515_vir_ani.tsv \
--out 250516_viral_clusters.tsv --min_ani 95 --min_tcov 85 --min_qcov 0



### Cant figure out why sourmash viru isnt working, cant find the right kmers
How similar are kmers between viral sequences, and how is the jaccard score compared to bacteria?

I shouldve made a snakefile

cdhit is not clustering things that should be clustered. dRep does cluster them. But dRep cannot handle so many sequences. Will try branchwater multisearch, followed by branchwater cluster, on average ANI column. Hopefully that would cluster more. 


In [ ]:
# need to first sketch the MAGs, manysketch

echo name,genome_filename,protein_filename > manysketch_all.csv
for i in *.fasta
do
  echo "$i,$(realpath "$i"),"
done >> /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/manysketch_all.csv


# then for sketching
srun --account=ctbrowngrp -p med2 -J skipm -t 2:00:00 -c 60 --mem=100gb --pty bash

sourmash scripts manysketch -p dna,k=21,k=31,k=51,scaled=100 \
-c 60 manysketch_99.csv -o MAGs.99.zip


In [ ]:
# make it into a rocksdb 
mamba activate branchwater-skipmer

srun --account=ctbrowngrp -p med2 -J skipm -t 2:00:00 -c 60 --mem=100gb --pty bash

sourmash index MAGs.99.rocksdb MAGs.99.dna.zip -F rocksdb


In [ ]:
# pairwise the MAGs
sourmash scripts pairwise \
MAGs.99.dna.zip -m DNA -k 51 \
-s 1000 -t 0.5 -o ../pairwise/MAGs.dna.k51.s1000.csv \
-c 60

sourmash scripts pairwise \
MAGs.99.dna.zip -m DNA -k 31 \
-s 100 -t 0.5 -o ../pairwise/MAGs.dna.k31.csv \
-c 60

sourmash scripts pairwise \
MAGs.99.dna.zip -m DNA -k 21 \
-s 100 -t 0.5 -o ../pairwise/MAGs.dna.k21.csv \
-c 60



In [ ]:
mamba activate branchwater-skipmer

srun --account=ctbrowngrp -p med2 -J skipm -t 2:00:00 -c 40 --mem=100gb --pty bash

sourmash scripts pairwise \
pig_votus.skipmer.zip -m skipm2n3 -k 21 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.skipm2n3.k21.csv \
-c 40 --write-all

sourmash scripts pairwise \
pig_votus.skipmer.zip -m skipm2n3 -k 27 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.skipm2n3.k27.csv \
-c 40 --write-all

sourmash scripts pairwise \
pig_votus.skipmer.zip -m skipm2n3 -k 24 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.skipm2n3.k24.csv \
-c 40 --write-all

sourmash scripts pairwise \
hq_virseqs.singleton.95.zip -m DNA -k 51 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.dna.k51.csv \
-c 40 --write-all

sourmash scripts pairwise \
hq_virseqs.singleton.95.zip -m DNA -k 31 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.dna.k31.csv \
-c 40 --write-all

sourmash scripts pairwise \
hq_virseqs.singleton.95.zip -m DNA -k 21 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.dna.k21.csv \
-c 40 --write-all

In [ ]:
dRep dereplicate ./drep --S_algorithm ANImf \
--ignoreGenomeQuality -l 500 -sa 0.95 -nc 0.85 \
-g ./250418_plasmid_sequences.genomad.fa.split/*.fa -p 48